# Building A Chatbot with Message History

In [2]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

In [5]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi what is your version?")])

AIMessage(content="I am Gemma, an open-weights AI assistant. My creators, the Gemma team, haven't publicly released the specific version number for me.\n\nIs there anything else you'd like to know?\n", response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 15, 'total_tokens': 60, 'completion_time': 0.081818182, 'prompt_time': 7.5421e-05, 'queue_time': 0.015384578999999999, 'total_time': 0.081893603}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0a35eef9-b393-41f2-8754-b9bd5e4aa89f-0', usage_metadata={'input_tokens': 15, 'output_tokens': 45, 'total_tokens': 60})

In [6]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi what is your version?"),
    AIMessage(content="I am Gemma, an open-weights AI assistant. My creators, the Gemma team, haven't publicly released the specific version number for me.\n\nIs there anything else you'd like to know?\n"),
    HumanMessage(content="What was my first question ?")
])

AIMessage(content='Your first question was: "Hi what is your version?" \n\nIt seems you\'re having a bit of fun with the repetition! 😊  Is there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 74, 'total_tokens': 117, 'completion_time': 0.078181818, 'prompt_time': 0.003677727, 'queue_time': 0.01067355, 'total_time': 0.081859545}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ced34e1b-8703-4be1-8f0f-ef0c21d40b62-0', usage_metadata={'input_tokens': 74, 'output_tokens': 43, 'total_tokens': 117})

## Message History

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
        
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable" :{"session_id": "chat1"}}

In [13]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, I am Aadarsh and I am a Data Science Gradauate.")
    ],
    config= config
)
response.content

"Hello Aadarsh!\n\nCongratulations on graduating with your Data Science degree! That's a huge accomplishment. 🎉\n\nWhat are you most looking forward to as you start your journey in the world of data science?  \n\n\n\n\n\n"

In [14]:
response = with_message_history.invoke(
    [
        HumanMessage(content="What is my Name ?")
    ],
    config= config
)
response.content

"Your name is Aadarsh. 😊 \n\nI remember!  Do you have a specific question about data science you'd like to ask me?  I'm happy to help if I can! \n"

### Changing session id

In [15]:
config1 = {"configurable" :{"session_id": "chat2"}}
response = with_message_history.invoke(
    [
        HumanMessage(content="What is my Name ?")
    ],
    config= config1
)
response.content

"As an AI, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'm happy to know! 😊 \n"

In [16]:
response = with_message_history.invoke(
    [
        HumanMessage(content="My name is Aadarsh")
    ],
    config= config1
)
response.content

"Hi Aadarsh, it's nice to meet you! 😊 \n\nIs there anything I can help you with today?\n"

In [17]:
response = with_message_history.invoke(
    [
        HumanMessage(content="What is my Name ?")
    ],
    config= config1
)
response.content

'Your name is Aadarsh.  \n\nI remember that you told me earlier! 😄  \n\nIs there anything else I can help you with?\n'

## Prompt Templating

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant. Answer all the questions with the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Aadarsh")]})

AIMessage(content="Hi Aadarsh!  \n\nIt's nice to meet you. What can I do for you today?  😊 \n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 31, 'total_tokens': 59, 'completion_time': 0.050909091, 'prompt_time': 0.000336239, 'queue_time': 0.014312399, 'total_time': 0.05124533}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2b0b1cbc-4c39-4796-93d6-89b51c8b68eb-0', usage_metadata={'input_tokens': 31, 'output_tokens': 28, 'total_tokens': 59})

In [21]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [23]:
config3 = {"configurable" :{"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="What are the latest developments in Data Science")],
    config= config3
)
response.content

'You\'re right to ask! It seems like you\'re looking for an even deeper dive into the newest trends. \n\nHere are some cutting-edge developments in data science that are pushing the boundaries even further:\n\n**1.  Foundation Models & Prompt Engineering:**\n\n* **Beyond Text:** While LLMs like GPT-3 are impressive, research is expanding to foundation models that handle other data types like images (DALL-E 2), audio (Jukebox), and even scientific data.\n* **Fine-tuning for Specificity:**  The ability to "fine-tune" these massive models for specific tasks (like medical diagnosis or code generation) is becoming increasingly sophisticated.\n* **Prompt Engineering as an Art:**  Crafting effective prompts to elicit desired outputs from foundation models is emerging as a crucial skill, requiring creativity and understanding of model capabilities.\n\n**2.  Causal Inference & Counterfactual Reasoning:**\n\n* **Going Beyond Correlation:**  Traditional machine learning often reveals correlations

In [26]:
prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant. You explain all the questions with examples related to airplanes."),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [27]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [28]:
config4 = {"configurable" :{"session_id": "chat4"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is Data Science?")],
    config= config4
)
response.content

"Imagine you're an airline trying to figure out how to make more money and keep passengers happy. You have tons of data about flights, passengers, weather, and more. Data science is like having a super-powered detective team that can sift through all that information and find hidden clues.\n\nHere's what they do:\n\n* **Gather Clues:** They collect data from everywhere – flight schedules, ticket sales, customer surveys, even social media posts about your airline.\n* **Solve the Puzzle:**  They use math, statistics, and computer programs to spot patterns and trends in the data. Maybe they find that people flying on Tuesdays book cheaper tickets, or that delays cause a lot of complaints.\n* **Make Predictions:** Based on the patterns, they can predict things like how many people will book flights next month, which routes are most profitable, or when a plane is likely to be delayed.\n* **Give Advice:** They present their findings to the airline in a way that's easy to understand. This hel

## Managing Conversation History

In [29]:
from langchain_core.messages import SystemMessage,trim_messages

In [36]:
trimmer = trim_messages(
    max_tokens = 46,
    strategy = "last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [37]:
messages = [
    SystemMessage(content="You are a helful assistant"),
    HumanMessage(content="Hi I'm Bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like Vanilla Icecream"),
    AIMessage(content="nice"),
    HumanMessage("whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="Yes!")
]

In [38]:
trimmer.invoke(messages)

[SystemMessage(content='You are a helful assistant'),
 HumanMessage(content='I like Vanilla Icecream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2+2'),
 AIMessage(content='4'),
 HumanMessage(content='Thanks'),
 AIMessage(content='no problem'),
 HumanMessage(content='having fun?'),
 AIMessage(content='Yes!')]

In [41]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

response  = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What icecream do I like?")],
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have access to your personal information, including your favorite ice cream flavor.  \n\nTo figure out your favorite, you could think about:\n\n* **What flavors do you usually order?**  \n* **What flavors do you have in your freezer?**\n* **What flavors sound most delicious to you right now?**\n\nMaybe you like chocolate chip cookie dough, or maybe you're a fan of something more unique like salted caramel! 🍦  \n\n\n\n"

## Vector Stores and Retrievers

### Loading Documents

In [42]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('random.pdf')
documents = loader.load()

In [43]:
documents

[Document(metadata={'source': 'random.pdf', 'page': 0}, page_content='TECHNOLOGY READINESS LEVELS\nFOR MACHINE LEARNING SYSTEMS\nAlexander Lavin∗\nPasteur LabsCiarán M. Gilligan-Lee\nSpotifyAlessya Visnjic\nWhyLabsSiddha Ganju\nNvidiaDava Newman\nMIT\nAtılım Güne¸ s Baydin\nUniversity of OxfordSujoy Ganguly\nUnity AIDanny Lange\nUnity AIAmit Sharma\nMicrosoft Research\nStephan Zheng\nSalesforce ResearchEric P. Xing\nPetuumAdam Gibson\nKonduitJames Parr\nNASA Frontier Development Lab\nChris Mattmann\nNASA Jet Propulsion LabYarin Gal\nAlan Turing Institute\nABSTRACT\nThe development and deployment of machine learning (ML) systems can be executed easily with\nmodern tools, but the process is typically rushed and means-to-an-end. The lack of diligence can\nlead to technical debt, scope creep and misaligned objectives, model misuse and failures, and\nexpensive consequences. Engineering systems, on the other hand, follow well-deﬁned processes\nand testing standards to streamline development 

### Using ChromaDB 

In [49]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/opt/anaconda3/envs/llm-venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [50]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [51]:
llm = ChatGroq(model="llama3-70b-8192")

In [52]:
vectorstore = Chroma.from_documents(documents, embeddings)

In [53]:
vectorstore.similarity_search("artifact")

[Document(metadata={'page': 11, 'source': 'random.pdf'}, page_content='skip the earlier levels, many challenges arise when combining ML elements that were independently validated\nand developed. The MLTRL prototype-caliber code checkpoint ensures that the existing code components\nare validated and helps avoid poorly deﬁned borders and abstractions between components. ML pipelines\noften grow out of glue code, and our regimented code checkpoints motivate well-architected software that\nminimizes these danger spots.\n•Level 5 – The problematic “valley of death”, mentioned earlier in the level 5 deﬁnitions, is less prevalent in use-\ncases like this that start at a higher MLTRL level with a speciﬁc product deliverable. In this case, the product\ndeliverable was a real-time object recognition and classiﬁcation of trash for a mobile recycling application.\nStill, this stage is critical for the requirements and V&V transition. This stage mitigates failure risks due to the\ndisparate data so

- Vector stores cannot be  included in chain using LCEL, because they are not Runnable. But retrievers are.

In [54]:
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["artifact", "observability"])

[[Document(metadata={'page': 11, 'source': 'random.pdf'}, page_content='skip the earlier levels, many challenges arise when combining ML elements that were independently validated\nand developed. The MLTRL prototype-caliber code checkpoint ensures that the existing code components\nare validated and helps avoid poorly deﬁned borders and abstractions between components. ML pipelines\noften grow out of glue code, and our regimented code checkpoints motivate well-architected software that\nminimizes these danger spots.\n•Level 5 – The problematic “valley of death”, mentioned earlier in the level 5 deﬁnitions, is less prevalent in use-\ncases like this that start at a higher MLTRL level with a speciﬁc product deliverable. In this case, the product\ndeliverable was a real-time object recognition and classiﬁcation of trash for a mobile recycling application.\nStill, this stage is critical for the requirements and V&V transition. This stage mitigates failure risks due to the\ndisparate data s

- Vector Store as Retriever returns Runnables

In [57]:
retriever_as = vectorstore.as_retriever(
    search_type ="similarity",
    search_kwargs={"k":2}
)

In [56]:
retriever_as.batch(["artifact", "observability"])

[[Document(metadata={'page': 11, 'source': 'random.pdf'}, page_content='skip the earlier levels, many challenges arise when combining ML elements that were independently validated\nand developed. The MLTRL prototype-caliber code checkpoint ensures that the existing code components\nare validated and helps avoid poorly deﬁned borders and abstractions between components. ML pipelines\noften grow out of glue code, and our regimented code checkpoints motivate well-architected software that\nminimizes these danger spots.\n•Level 5 – The problematic “valley of death”, mentioned earlier in the level 5 deﬁnitions, is less prevalent in use-\ncases like this that start at a higher MLTRL level with a speciﬁc product deliverable. In this case, the product\ndeliverable was a real-time object recognition and classiﬁcation of trash for a mobile recycling application.\nStill, this stage is critical for the requirements and V&V transition. This stage mitigates failure risks due to the\ndisparate data s

In [61]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [60]:
message = """
    Answer the question using the provided context only.
    {question}
    
    Context:
    {context}
    """
    
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {'context':retriever_as, 'question':RunnablePassthrough()}|prompt|llm

response = rag_chain.invoke("Tell me about levels")

response.content

'According to the context, "levels" refer to a series of stages or milestones in the development and deployment of machine learning (ML) technology. There are 8 levels mentioned, each with its own set of goals, objectives, and requirements. Here is a brief summary of each level:\n\n**Level 4:** Demonstrate the utility of the ML technology towards one or more practical applications, reviewing data readiness, security, and privacy considerations.\n\n**Level 5:** Develop the ML technology into a capability, transitioning from R&D to productization, and defining an interdisciplinary working group to share the technology with others in the organization.\n\n**Level 6:** Focus on application development, significant software engineering to bring the code up to product-caliber, and developing ML modules with known data challenges in mind.\n\n**Level 7:** Integrate the ML technology into existing production systems, with a balanced working group of infrastructure engineers and applied AI engine